In [1]:

list_destination=["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [2]:
import scrapy
import os
import datetime
import json
import pandas as pd

In [3]:
os.getcwd()

'c:\\Users\\nicol\\Desktop\\Kayakproject'

In [4]:
os.chdir("main")
os.getcwd()

'c:\\Users\\nicol\\Desktop\\Kayakproject\\main'

In [5]:
for destination in list_destination:
    os.environ["listdestination"]=destination
    !scrapy crawl bookingmain

2022-05-02 16:14:46 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-02 16:14:46 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-10-10.0.19043-SP0
2022-05-02 16:14:46 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-05-02 16:14:46 [scrapy.extensions.telnet] INFO: Telnet Password: fff953bbd5cb1343
2022-05-02 16:14:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-05-02 16:14:47 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloader

^C


2022-05-02 16:16:06 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-02 16:16:06 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-10-10.0.19043-SP0
2022-05-02 16:16:06 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-05-02 16:16:06 [scrapy.extensions.telnet] INFO: Telnet Password: d87915bbef8b96fa
2022-05-02 16:16:06 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-05-02 16:16:07 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloader

In [6]:
def clean_nones(value):
    """
    Recursively remove all None values from dictionaries and lists, and returns
    the result as a new dictionary or list.
    """
    if isinstance(value, list):
        return [clean_nones(x) for x in value if x is not None]
    elif isinstance(value, dict):
        return {
            key: clean_nones(val)
            for key, val in value.items()
            if val is not None
        }
    else:
        return value

In [7]:
os.chdir("results")

In [8]:
os.getcwd()

'c:\\Users\\nicol\\Desktop\\Kayakproject\\main\\results'

In [9]:
for files in os.listdir():
    with open(files) as json_file:
        data = json.load(json_file)
        datacleaned=clean_nones(data)
        datacleaned2=list(filter(None, datacleaned))
        with open(files, 'w') as f:
            json.dump(datacleaned2, f)

In [10]:
os.chdir("..")
os.chdir("..")
os.chdir("detailed")
os.getcwd()


'c:\\Users\\nicol\\Desktop\\Kayakproject\\main'

In [19]:
os.getcwd()

'c:\\Users\\nicol\\Desktop\\Kayakproject\\detailed'

In [27]:
pathresults= os.path.dirname(os.getcwd())+"\\main\\results"
for file in os.listdir(pathresults):
    os.environ["files"]=str(pathresults+"\\"+file)
    !scrapy crawl bookingindividual -o xxx.json

2022-05-02 16:30:45 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-02 16:30:45 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-10-10.0.19043-SP0
2022-05-02 16:30:45 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-05-02 16:30:45 [scrapy.extensions.telnet] INFO: Telnet Password: 6c5471312aaad27d
2022-05-02 16:30:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-05-02 16:30:46 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloader

In [30]:
os.getcwd()

'c:\\Users\\nicol\\Desktop\\Kayakproject\\detailed'

In [35]:
#merge all files

dataframemain= pd.DataFrame(columns = ["name","url","Rating"])


pathresults= os.path.dirname(os.getcwd())+"\\main\\results"
for file in os.listdir(pathresults):
    with open(os.path.dirname(os.getcwd())+"\\main\\results\\"+file) as json_file:
        data = json.load(json_file)
        dataframemain=pd.concat([dataframemain,pd.DataFrame(data)])




In [42]:
dataframemain.describe()

,name,url,Rating
count,883,883,861
unique,875,883,48
top,Hôtel Central,https://www.booking.com/hotel/fr/canal-aigues-...,"8,1"
freq,2,1,48


In [37]:
dataframedetailed= pd.DataFrame(columns = ["url","description","bbox","Destination"])


pathresults= os.getcwd()+"\\resultsdetailed"
for file in os.listdir(pathresults):
    with open(pathresults+"\\"+file) as json_file:
        data = json.load(json_file)
        dataframedetailed=pd.concat([dataframedetailed,pd.DataFrame(data)])

In [56]:
dataframedetailed.head()

,url,description,bbox,Destination
0,https://www.booking.com/hotel/fr/residence-oda...,[Vous pouvez bénéficier d'une réduction Genius...,"4.15440270440472,43.5561285834269,4.2040892955...",Aigues Mortes
1,https://www.booking.com/hotel/fr/le-dit-vin-se...,[Vous pouvez bénéficier d'une réduction Genius...,"4.16427272565645,43.5488035834269,4.2139532743...",Aigues Mortes
2,https://www.booking.com/hotel/fr/le-royal-hote...,[Featuring free WiFi and a seasonal outdoor sw...,"4.17297379558578,43.5584304713006,4.2226622861...",Aigues Mortes
3,https://www.booking.com/hotel/fr/canal-aigues-...,[Vous pouvez bénéficier d'une réduction Genius...,"4.16998185644106,43.5550221035115,4.2196675347...",Aigues Mortes
4,https://www.booking.com/hotel/fr/saint-louis-a...,[Vous pouvez bénéficier d'une réduction Genius...,"4.16450314127171,43.549254254998,4.21418406165...",Aigues Mortes


In [52]:
dataframedetailed["url"]=dataframedetailed["url"].apply(lambda x: x+"#hotelTmpl")

In [53]:
dfbbox=pd.merge(dataframemain,dataframedetailed,on="url")

In [55]:
dfbbox.describe()

,name,url,Rating,description,bbox,Destination
count,883,883,861,883,883,883
unique,875,883,48,877,875,35
top,Hôtel Central,https://www.booking.com/hotel/fr/canal-aigues-...,"8,1",[Vous pouvez bénéficier d'une réduction Genius...,"7.33500722464803,48.0601503371431,7.3888857086...",Toulouse
freq,2,1,48,2,2,27


In [ ]:
#covnert bbox to gps

In [71]:
dfbbox["Lat"]=dfbbox["bbox"].apply(lambda x: float(x.split(",")[1])/2+float(x.split(",")[3])/2)
dfbbox["Lon"]=dfbbox["bbox"].apply(lambda x: float(x.split(",")[0])/2+float(x.split(",")[2])/2)


In [73]:
dfbbox.head()

,name,url,Rating,description,bbox,Destination,Lat,Lon
0,Hotel Canal Aigues Mortes,https://www.booking.com/hotel/fr/canal-aigues-...,"8,6",[Vous pouvez bénéficier d'une réduction Genius...,"4.16998185644106,43.5550221035115,4.2196675347...",Aigues Mortes,43.572988,4.194825
1,La Villa Mazarin,https://www.booking.com/hotel/fr/la-villa-maza...,"9,1",[Vous pouvez bénéficier d'une réduction Genius...,"4.16691241748164,43.5470212030719,4.2165914962...",Aigues Mortes,43.564987,4.191752
2,Maison des Croisades,https://www.booking.com/hotel/fr/des-croisades...,"8,9",[Vous pouvez bénéficier d'une réduction Genius...,"4.1634570066567,43.5510319834269,4.21313939334...",Aigues Mortes,43.568997,4.188298
3,Hôtel Saint Louis,https://www.booking.com/hotel/fr/saint-louis-a...,"8,4",[Vous pouvez bénéficier d'une réduction Genius...,"4.16450314127171,43.549254254998,4.21418406165...",Aigues Mortes,43.567220,4.189344
4,Noemys Aigues-Mortes - ex Mona Lisa Royal Hôtel,https://www.booking.com/hotel/fr/le-royal-hote...,"6,9",[Featuring free WiFi and a seasonal outdoor sw...,"4.17297379558578,43.5584304713006,4.2226622861...",Aigues Mortes,43.576396,4.197818


In [76]:
os.chdir("..")

dfbbox.to_csv("Hotels.csv")